# Carlos Llompart López
Inteligencia Artificial

Práctica 3. FIFA practice

Miquel Miró Nicolau

https://github.com/cllompart/PRACTICA-FIFA



# Fifa practice

We want to predict the *value* of the players using the information of the player.

## Import libraries

First of all we import the libraries we're gonna use.

## Read the data

To read the data we're gonna use ``pandas`` and an utility from the library
``os``.


In [1]:
import os

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import pandas as pd
import numpy as np

Creamos una base que contrendrá la constrrucción del dataset con los datos de entrenamiento relativos a la variable objetivo 'Valor'. De esta manera, comprobaremos como mejoran esos datos en nuestro modelo de predicción.

In [2]:
df = pd.read_csv(os.path.join("..", "in", "fifa.csv"))
base = df
base.head()

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,...,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,...,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,...,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


Ya tenemos los jugadores cargados en el data frame

Now we select one columns. The result will be an object from the ``pandas`` library a ``series`` object. This object can be converted to a more standard ``numpy`` array.

In [3]:
name = base["Name"]
name 

0                  L. Messi
1         Cristiano Ronaldo
2                 Neymar Jr
3                    De Gea
4              K. De Bruyne
                ...        
18202          J. Lundstram
18203    N. Christoffersson
18204             B. Worman
18205        D. Walker-Rice
18206             G. Nugent
Name: Name, Length: 18207, dtype: object

## Data analysis 

First of all we want to known what is happening to the data. To do so we increase the number of columns to show. After that we're gonna use a special instruction that will show us the data structure.



In [4]:
pd.set_option('display.max_columns', None)

In [5]:
base.head()

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,€110.5M,€565K,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,Yes,RF,10.0,"Jul 1, 2004",NaN,2021,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,€77M,€405K,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,Yes,ST,7.0,"Jul 10, 2018",NaN,2022,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,https://cdn.sofifa.org/teams/2/light/73.png,€118.5M,€290K,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,Yes,LW,10.0,"Aug 3, 2017",NaN,2022,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,https://cdn.sofifa.org/teams/2/light/11.png,€72M,€260K,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,Yes,GK,1.0,"Jul 1, 2011",NaN,2020,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,https://cdn.sofifa.org/teams/2/light/10.png,€102M,€355K,2281,Right,4.0,5.0,4.0,High/ High,Normal,Yes,RCM,7.0,"Aug 30, 2015",NaN,2023,5'11,154lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


In [6]:
base.describe()

,Unnamed: 0,ID,Age,Overall,Potential,Special,International Reputation,Weak Foot,Skill Moves,Jersey Number,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes
count,18207.000000,18207.000000,18207.000000,18207.000000,18207.000000,18207.000000,18159.000000,18159.000000,18159.000000,18147.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000
mean,9103.000000,214298.338606,25.122206,66.238699,71.307299,1597.809908,1.113222,2.947299,2.361308,19.546096,49.734181,45.550911,52.298144,58.686712,42.909026,55.371001,47.170824,42.863153,52.711933,58.369459,64.614076,64.726967,63.503607,61.836610,63.966573,55.460047,65.089432,63.219946,65.311967,47.109973,55.868991,46.698276,49.958478,53.400903,48.548598,58.648274,47.281623,47.697836,45.661435,16.616223,16.391596,16.232061,16.388898,16.710887
std,5256.052511,29965.244204,4.669943,6.908930,6.136496,272.586016,0.394031,0.660456,0.756164,15.947765,18.364524,19.525820,17.379909,14.699495,17.694408,18.910371,18.395264,17.478763,15.327870,16.686595,14.927780,14.649953,14.766049,9.010464,14.136166,17.237958,11.820044,15.894741,12.557000,19.260524,17.367967,20.696909,19.529036,14.146881,15.704053,11.436133,19.904397,21.664004,21.289135,17.695349,16.906900,16.502864,17.034669,17.955119
min,0.000000,16.000000,16.000000,46.000000,48.000000,731.000000,1.000000,1.000000,1.000000,1.000000,5.000000,2.000000,4.000000,7.000000,4.000000,4.000000,6.000000,3.000000,9.000000,5.000000,12.000000,12.000000,14.000000,21.000000,16.000000,2.000000,15.000000,12.000000,17.000000,3.000000,11.000000,3.000000,2.000000,10.000000,5.000000,3.000000,3.000000,2.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,4551.500000,200315.500000,21.000000,62.000000,67.000000,1457.000000,1.000000,3.000000,2.000000,8.000000,38.000000,30.000000,44.000000,54.000000,30.000000,49.000000,34.000000,31.000000,43.000000,54.000000,57.000000,57.000000,55.000000,56.000000,56.000000,45.000000,58.000000,56.000000,58.000000,33.000000,44.000000,26.000000,38.000000,44.000000,39.000000,51.000000,30.000000,27.000000,24.000000,8.000000,8.000000,8.000000,8.000000,8.000000
50%,9103.000000,221759.000000,25.000000,66.000000,71.000000,1635.000000,1.000000,3.000000,2.000000,17.000000,54.000000,49.000000,56.000000,62.000000,44.000000,61.000000,48.000000,41.000000,56.000000,63.000000,67.000000,67.000000,66.000000,62.000000,66.000000,59.000000,66.000000,66.000000,67.000000,51.000000,59.000000,52.000000,55.000000,55.000000,49.000000,60.000000,53.000000,55.000000,52.000000,11.000000,11.000000,11.000000,11.000000,11.000000
75%,13654.500000,236529.500000,28.000000,71.000000,75.000000,1787.000000,1.000000,3.000000,3.000000,26.000000,64.000000,62.000000,64.000000,68.000000,57.000000,68.000000,62.000000,57.000000,64.000000,69.000000,75.000000,75.000000,74.000000,68.000000,74.000000,68.000000,73.000000,74.000000,74.000000,62.000000,69.000000,64.000000,64.000000,64.000000,60.000000,67.000000,64.000000,66.000000,64.000000,14.000000,14.000000,14.000000,14.000000,14.000000
max,18206.000000,246620.000000,45.000000,94.000000,95.000000,2346.000000,5.000000,5.000000,5.000000,99.000000,93.000000,95.000000,94.000000,93.000000,90.000000,97.000000,94.000000,94.000000,93.000000,96.000000,97.000000,96.000000,96.000000,96.000000,96.000000,95.000000,95.000000,96.000000,97.000000,94.000

We can see that the two first columns do not provide any meaningful information so we can delete them.

In [7]:
base = base.iloc[:, 2:]
base

,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,€110.5M,€565K,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,Yes,RF,10.0,"Jul 1, 2004",NaN,2021,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,€77M,€405K,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,Yes,ST,7.0,"Jul 10, 2018",NaN,2022,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,https://cdn.sofifa.org/teams/2/light/73.png,€118.5M,€290K,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,Yes,LW,10.0,"Aug 3, 2017",NaN,2022,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,https://cdn.sofifa.org/teams/2/light/11.png,€72M,€260K,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,Yes,GK,1.0,"Jul 1, 2011",NaN,2020,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,https://cdn.sofifa.org/teams/2/light/10.png,€102M,€355K,2281,Right,4.0,5.0,4.0,High/ High,Normal,Yes,RCM,7.0,"Aug 30, 2015",NaN,2023,5'11,154lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,J. Lundstram,19,https://cdn.sofifa.org/players/

Con la funcion describe(), fijándonos en la desviación típica, podemos comprobar que existen datos que no nos proporcionan información importante para evaluar el valor de un jugador. Esta información solo provocará que empeore nuestro modelo de entrenamiento Por lo tanto, eliminaremos las columnas : Nombre, Foto, Bandera, Logo del Club,Cara Real y Dorsal.

In [8]:
base.drop(['Name','Photo','Flag','Club Logo','Real Face','Jersey Number'], axis = 1,inplace=True)
base

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Position,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,31,Argentina,94,94,FC Barcelona,€110.5M,€565K,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,RF,"Jul 1, 2004",NaN,2021,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,33,Portugal,94,94,Juventus,€77M,€405K,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,ST,"Jul 10, 2018",NaN,2022,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,26,Brazil,92,93,Paris Saint-Germain,€118.5M,€290K,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,LW,"Aug 3, 2017",NaN,2022,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,27,Spain,91,93,Manchester United,€72M,€260K,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,GK,"Jul 1, 2011",NaN,2020,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,27,Belgium,91,92,Manchester City,€102M,€355K,2281,Right,4.0,5.0,4.0,High/ High,Normal,RCM,"Aug 30, 2015",NaN,2023,5'11,154lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,19,England,47,65,Crewe Alexandra,€60K,€1K,1307,Right,1.0,2.0,2.0,Medium/ Medium,Lean,CM,"May 3, 2017",NaN,2019,5'9,134lbs,42+2,42+2,42+2,44+2,44+2,44+2,44+2,44+2,45+2,45+2,45+2,44+2,45+2,45+2,45+2,44+2,44+2,45+2,45+2,45+2,44+2,45+2,45+2,45+2,45+2,45+2,34.0,38.0,40.0,49.0,25.0,42.0,30.0,34.0,45.0,43.0,54.0,57.0,60.0,49.0,76.0,43.0,55.0,40.0,47.0,38.0,46.0,46.0,39.0,52.0,43.0,45.0,40.0,48.0,47.0,10.0,13.0,7.0,8.0,9.0,€143K
18203,19,Sweden,47,63,Trelleborgs FF,€60K,€1K,1098,Right,1.0,2.0,2.0,Medium/ Medium,Normal,ST,"Mar 19, 2018",NaN,2020,6'3,170lbs,45+2,45+2,45+2,39+2,42+2,42+2,42+2,39+2,40+2,40+2,40+2,38+2,35+2,35+2,35+2,38+2,30+2,31+2,31+2,31+2,30+2,29+2,32+2,32+2,32+2,29+2,23.0,52.0,52.0,43.0,36.0,39.0,32.0,20.0,25.0,40.0,41.0,39.0,38.0,40.0,52.0,41.0,47.0,43.0,67.0,42.0,47.0,16.0,46.0,33.0,43.0,42.0,22.0,15.0,19.0,10.0,9.0,9.0,5.0,12.0,€11

Por otro lado, las características Joined and Loaned From son más importantes que las anteriores a la hora de construir el modelo de predicción, pero no lo suficiente para contar con ellas. Por lo tanto, las eliminaremos

In [9]:
base.drop(['Joined','Loaned From','Contract Valid Until'], axis = 1,inplace=True)
base.head()

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Position,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,31,Argentina,94,94,FC Barcelona,€110.5M,€565K,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,RF,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,33,Portugal,94,94,Juventus,€77M,€405K,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,ST,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,26,Brazil,92,93,Paris Saint-Germain,€118.5M,€290K,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,LW,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,27,Spain,91,93,Manchester United,€72M,€260K,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,GK,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,27,Belgium,91,92,Manchester City,€102M,€355K,2281,Right,4.0,5.0,4.0,High/ High,Normal,RCM,5'11,154lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


## Gestión de valores no númericos

La mayoría de datos no están en el formato correcto, en float y contienen NaNs por ello, habrá que tratarlos.

Observamos que hay valores que no están en el formato correcto, entre ellos : Valor, Salario, Cláusula de recisión,  Peso, Altura

El valor, salario y cláusula de recisión están en formato monetario, en € concretamente. Por otro lado,  las abreviaturas 'K' y 'M' representan mil euros y millón de euros respectivamente.

Esta función nos transforma value a float. 

In [10]:
def value_to_float(x):
    """
    From K and M to float.
    
    """
    x = x.replace('€', '')
    ret_val = 0.0
    
    if type(x) == float or type(x) == int:
        ret_val = x
    if 'K' in x:
        if len(x) > 1:
            ret_val = float(x.replace('K', ''))
        ret_val = ret_val *1000
    if 'M' in x:
        if len(x) > 1:
            ret_val = float(x.replace('M', ''))
        ret_val = ret_val * 1000000.0
    return ret_val

Esta función nos transforma wage a float.

In [11]:
def wage_to_float(x):
    """
    From K and M to float.
    
    """
    x = x.replace('€', '')
    ret_val = 0.0
    
    if type(x) == float or type(x) == int:
        ret_val = x
    if 'K' in x:
        if len(x) > 1:
            ret_val = float(x.replace('K', ''))
        ret_val = ret_val *1000
    if 'M' in x:
        if len(x) > 1:
            ret_val = float(x.replace('M', ''))
        ret_val = ret_val * 1000000.0
    return ret_val

Esta función nos transforma release clause a float.

In [12]:
def releaseclause_to_float(x):
    """
    From K and M to float.
    
    """
    x = x.replace('€', '')
    ret_val = 0.0
    
    if type(x) == float or type(x) == int:
        ret_val = x
    if 'K' in x:
        if len(x) > 1:
            ret_val = float(x.replace('K', ''))
        ret_val = ret_val *1000
    if 'M' in x:
        if len(x) > 1:
            ret_val = float(x.replace('M', ''))
        ret_val = ret_val * 1000000.0
    return ret_val

Las funciones funcionaron bien en las columnas Valor y Salario,como se puede comprobar abajo.Por otro lado, en la clausula de recisión me dió error pues contiene valores NaN.

In [13]:
base["Value"] = base["Value"].apply(value_to_float)
base["Wage"] = base["Wage"].apply(wage_to_float)
base.head()

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Position,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,31,Argentina,94,94,FC Barcelona,110500000.0,565000.0,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,RF,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,33,Portugal,94,94,Juventus,77000000.0,405000.0,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,ST,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,26,Brazil,92,93,Paris Saint-Germain,118500000.0,290000.0,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,LW,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,27,Spain,91,93,Manchester United,72000000.0,260000.0,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,GK,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,27,Belgium,91,92,Manchester City,102000000.0,355000.0,2281,Right,4.0,5.0,4.0,High/ High,Normal,RCM,5'11,154lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


Se puede comprobar con esta función.

In [14]:
base.loc[base['Release Clause'].isnull()]

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Position,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
28,26,Colombia,88,89,FC Bayern München,69500000.0,315000.0,2171,Left,4.0,3.0,4.0,Medium/ Medium,Normal,LAM,5'11,172lbs,80+3,80+3,80+3,84+3,83+3,83+3,83+3,84+3,85+3,85+3,85+3,83+3,81+3,81+3,81+3,83+3,69+3,68+3,68+3,68+3,69+3,65+3,58+3,58+3,58+3,65+3,90.0,83.0,62.0,89.0,90.0,85.0,89.0,86.0,83.0,90.0,73.0,67.0,83.0,85.0,76.0,86.0,54.0,70.0,68.0,92.0,64.0,55.0,80.0,89.0,81.0,87.0,52.0,41.0,44.0,15.0,15.0,15.0,5.0,14.0,NaN
38,30,Argentina,88,88,Milan,57000000.0,245000.0,1965,Right,4.0,4.0,3.0,High/ Medium,Normal,LS,6'1,196lbs,85+3,85+3,85+3,81+3,83+3,83+3,83+3,81+3,80+3,80+3,80+3,78+3,71+3,71+3,71+3,78+3,55+3,53+3,53+3,53+3,55+3,51+3,48+3,48+3,48+3,51+3,68.0,92.0,80.0,75.0,90.0,84.0,74.0,62.0,59.0,85.0,73.0,73.0,75.0,86.0,69.0,86.0,79.0,70.0,85.0,80.0,50.0,20.0,92.0,74.0,70.0,86.0,35.0,22.0,18.0,5.0,12.0,7.0,5.0,10.0,NaN
91,29,Brazil,85,85,Guangzhou Evergrande Taobao FC,37000000.0,235000.0,2245,Right,3.0,3.0,3.0,High/ High,Lean,LDM,6'0,157lbs,81+3,81+3,81+3,79+3,81+3,81+3,81+3,79+3,81+3,81+3,81+3,79+3,82+3,82+3,82+3,79+3,78+3,81+3,81+3,81+3,78+3,78+3,80+3,80+3,80+3,78+3,64.0,81.0,85.0,84.0,75.0,82.0,66.0,72.0,80.0,84.0,73.0,73.0,69.0,84.0,66.0,83.0,90.0,92.0,81.0,80.0,81.0,81.0,84.0,78.0,71.0,80.0,75.0,81.0,73.0,16.0,16.0,11.0,6.0,8.0,NaN
166,24,Brazil,83,90,Guangzhou Evergrande Taobao FC,36500000.0,18000.0,2142,Left,2.0,4.0,4.0,High/ Medium,Normal,CAM,6'3,176lbs,80+2,80+2,80+2,81+2,81+2,81+2,81+2,81+2,81+2,81+2,81+2,80+2,79+2,79+2,79+2,80+2,69+2,69+2,69+2,69+2,69+2,67+2,64+2,64+2,64+2,67+2,78.0,80.0,77.0,81.0,76.0,82.0,86.0,87.0,79.0,84.0,77.0,76.0,76.0,79.0,65.0,84.0,58.0,76.0,72.0,88.0,67.0,58.0,86.0,81.0,71.0,74.0,55.0,62.0,42.0,13.0,11.0,13.0,12.0,10.0,NaN
176,24,Croatia,83,89,Chelsea,35000000.0,165000.0,2115,Right,3.0,4.0,4.0,High/ High,Normal,LCM,5'9,172lbs,71+3,71+3,71+3,79+3,78+3,78+3,78+3,79+3,81+3,81+3,81+3,80+3,82+3,82+3,82+3,80+3,76+3,78+3,78+3,78+3,76+3,74+3,69+3,69+3,69+3,74+3,73.0,62.0,42.0,87.0,76.0,88.0,78.0,68.0,82.0,89.0,84.0,74.0,81.0,70.0,85.0,79.0,63.0,82.0,65.0,75.0,77.0,78.0,69.0,88.0,59.0,72.0,69.0,75.0,63.0,9.0,5.0,6.0,7.0,7.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17979,21,China PR,51,60,Guizhou Hengfeng FC,60000.0,2000.0,1459,Right,1.0,2.0,2.0,Medium/ Medium,Normal,CM,6'0,172lbs,49+2,49+2,49+2,50+2,50+2,50+2,50+2,50+2,50+2,50+2,50+2,51+2,50+2,50+2,50+2,51+2,50+2,50+2,50+2,50+2,50+2,50+2,49+2,49+2,49+2,50+2,36.0,30.0,48.0,57.0,32.0,45.0,37.0,38.0,55.0,50.0,71.0,76.0,75.0,55.0,58.0,55.0,60.0,62.0,61.0,33.0,58.0,41.0,53.0,47.0,36.0,43.0,47.0,41.0,46.0,12.0,12.0,10.0,13.0,9.0,NaN
18026,21,China PR,50,59,Guizhou Hengfeng FC,50000.0,2000.0,1440,Right,1.0,2.0,2.0,Medium/ Medium,Lean,LM,5'10,143lbs,44+2,44+2,44+2,47+2,46+2,46+2,46+2,47+2,48+2,48+2,48+2,49+2,48+2,48+2,48+2,49+2,50+2,50+2,50+2,50+2,50+2,51+2,50+2,50+2,50+2,51+2,41.0,28.0,50.0,54.0,26.0,50.0,39.0,40.0,55.0,46.0,61.0,77.0,76.0,50.0,74.0,47.0,63.0,50.0,52.0,29.0,56.0,50.0,37.0,49.0,42.0,38.0,46.0,48.0,51.0,15.0,8.0,10.0,13.0,7.0,NaN
18031,20,China PR,50,61,Stabæk Fotball,40000.0,2000.0,1278,Right,1.0,3.0,2.0,Medium/ Medium,Normal,RB,5'7,143lbs,37+2,37+2,37+2

Como gestioné esos NaNs? Al haber una gran cantidad de jugadores con esta característica y sobretodo por ser relativa al valor, no era una opción viable eliminarlos.Por ello, para resolverlo, sustituí(imputé) los NaNs por 0

In [15]:
base["Release Clause"].fillna('0€', inplace=True)

Comprobamos como la función funciona correctamente, pues en estos jugadores la característica Release Clause es igual a 0 €

In [16]:
base.loc[base['Release Clause'] == '0€']

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Position,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
28,26,Colombia,88,89,FC Bayern München,69500000.0,315000.0,2171,Left,4.0,3.0,4.0,Medium/ Medium,Normal,LAM,5'11,172lbs,80+3,80+3,80+3,84+3,83+3,83+3,83+3,84+3,85+3,85+3,85+3,83+3,81+3,81+3,81+3,83+3,69+3,68+3,68+3,68+3,69+3,65+3,58+3,58+3,58+3,65+3,90.0,83.0,62.0,89.0,90.0,85.0,89.0,86.0,83.0,90.0,73.0,67.0,83.0,85.0,76.0,86.0,54.0,70.0,68.0,92.0,64.0,55.0,80.0,89.0,81.0,87.0,52.0,41.0,44.0,15.0,15.0,15.0,5.0,14.0,0€
38,30,Argentina,88,88,Milan,57000000.0,245000.0,1965,Right,4.0,4.0,3.0,High/ Medium,Normal,LS,6'1,196lbs,85+3,85+3,85+3,81+3,83+3,83+3,83+3,81+3,80+3,80+3,80+3,78+3,71+3,71+3,71+3,78+3,55+3,53+3,53+3,53+3,55+3,51+3,48+3,48+3,48+3,51+3,68.0,92.0,80.0,75.0,90.0,84.0,74.0,62.0,59.0,85.0,73.0,73.0,75.0,86.0,69.0,86.0,79.0,70.0,85.0,80.0,50.0,20.0,92.0,74.0,70.0,86.0,35.0,22.0,18.0,5.0,12.0,7.0,5.0,10.0,0€
91,29,Brazil,85,85,Guangzhou Evergrande Taobao FC,37000000.0,235000.0,2245,Right,3.0,3.0,3.0,High/ High,Lean,LDM,6'0,157lbs,81+3,81+3,81+3,79+3,81+3,81+3,81+3,79+3,81+3,81+3,81+3,79+3,82+3,82+3,82+3,79+3,78+3,81+3,81+3,81+3,78+3,78+3,80+3,80+3,80+3,78+3,64.0,81.0,85.0,84.0,75.0,82.0,66.0,72.0,80.0,84.0,73.0,73.0,69.0,84.0,66.0,83.0,90.0,92.0,81.0,80.0,81.0,81.0,84.0,78.0,71.0,80.0,75.0,81.0,73.0,16.0,16.0,11.0,6.0,8.0,0€
166,24,Brazil,83,90,Guangzhou Evergrande Taobao FC,36500000.0,18000.0,2142,Left,2.0,4.0,4.0,High/ Medium,Normal,CAM,6'3,176lbs,80+2,80+2,80+2,81+2,81+2,81+2,81+2,81+2,81+2,81+2,81+2,80+2,79+2,79+2,79+2,80+2,69+2,69+2,69+2,69+2,69+2,67+2,64+2,64+2,64+2,67+2,78.0,80.0,77.0,81.0,76.0,82.0,86.0,87.0,79.0,84.0,77.0,76.0,76.0,79.0,65.0,84.0,58.0,76.0,72.0,88.0,67.0,58.0,86.0,81.0,71.0,74.0,55.0,62.0,42.0,13.0,11.0,13.0,12.0,10.0,0€
176,24,Croatia,83,89,Chelsea,35000000.0,165000.0,2115,Right,3.0,4.0,4.0,High/ High,Normal,LCM,5'9,172lbs,71+3,71+3,71+3,79+3,78+3,78+3,78+3,79+3,81+3,81+3,81+3,80+3,82+3,82+3,82+3,80+3,76+3,78+3,78+3,78+3,76+3,74+3,69+3,69+3,69+3,74+3,73.0,62.0,42.0,87.0,76.0,88.0,78.0,68.0,82.0,89.0,84.0,74.0,81.0,70.0,85.0,79.0,63.0,82.0,65.0,75.0,77.0,78.0,69.0,88.0,59.0,72.0,69.0,75.0,63.0,9.0,5.0,6.0,7.0,7.0,0€
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17979,21,China PR,51,60,Guizhou Hengfeng FC,60000.0,2000.0,1459,Right,1.0,2.0,2.0,Medium/ Medium,Normal,CM,6'0,172lbs,49+2,49+2,49+2,50+2,50+2,50+2,50+2,50+2,50+2,50+2,50+2,51+2,50+2,50+2,50+2,51+2,50+2,50+2,50+2,50+2,50+2,50+2,49+2,49+2,49+2,50+2,36.0,30.0,48.0,57.0,32.0,45.0,37.0,38.0,55.0,50.0,71.0,76.0,75.0,55.0,58.0,55.0,60.0,62.0,61.0,33.0,58.0,41.0,53.0,47.0,36.0,43.0,47.0,41.0,46.0,12.0,12.0,10.0,13.0,9.0,0€
18026,21,China PR,50,59,Guizhou Hengfeng FC,50000.0,2000.0,1440,Right,1.0,2.0,2.0,Medium/ Medium,Lean,LM,5'10,143lbs,44+2,44+2,44+2,47+2,46+2,46+2,46+2,47+2,48+2,48+2,48+2,49+2,48+2,48+2,48+2,49+2,50+2,50+2,50+2,50+2,50+2,51+2,50+2,50+2,50+2,51+2,41.0,28.0,50.0,54.0,26.0,50.0,39.0,40.0,55.0,46.0,61.0,77.0,76.0,50.0,74.0,47.0,63.0,50.0,52.0,29.0,56.0,50.0,37.0,49.0,42.0,38.0,46.0,48.0,51.0,15.0,8.0,10.0,13.0,7.0,0€
18031,20,China PR,50,61,Stabæk Fotball,40000.0,2000.0,1278,Right,1.0,3.0,2.0,Medium/ Medium,Normal,RB,5'7,143lbs,37+2,37+2,37+2,42+2,4

Ahora si podemos aplicar la función que transforma los valores monetarios de la cláusula de recisión a float

In [17]:
base["Release Clause"] = base["Release Clause"].apply(wage_to_float)

Comprobamos que en la última columna la característica Cláusula de reicisión es float

In [18]:
base.head()

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Position,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,31,Argentina,94,94,FC Barcelona,110500000.0,565000.0,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,RF,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,226500000.0
1,33,Portugal,94,94,Juventus,77000000.0,405000.0,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,ST,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,127100000.0
2,26,Brazil,92,93,Paris Saint-Germain,118500000.0,290000.0,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,LW,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,228100000.0
3,27,Spain,91,93,Manchester United,72000000.0,260000.0,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,GK,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,138600000.0
4,27,Belgium,91,92,Manchester City,102000000.0,355000.0,2281,Right,4.0,5.0,4.0,High/ High,Normal,RCM,5'11,154lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,196400000.0


Ahora me fijo en otras características que tampoco están en el formato adecuado como el peso y la altura que pueden ser importantes a la hora de predecir el valor del jugador. Por ello, las transformaremos a float con estas funciones.

In [19]:
def height_to_float(x):
    """
    x'x -> x.x
    """
    x = x.split("'")
    
    if(len(x[1])==1): x = float(x[0]) + float(x[1])/10
    else:
        y = float(x[1])
        y = y/100
        x = float(x[0])+y

    return x

In [20]:
base.loc[base['Height'].isnull()]

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Position,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
13236,33,Scotland,62,62,Rochdale,120000.0,1000.0,1510,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
13237,29,Nicaragua,62,62,Boyacá Chicó FC,300000.0,1000.0,1532,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
13238,35,England,62,62,Notts County,140000.0,3000.0,1573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
13239,20,Italy,62,72,Brescia,425000.0,1000.0,1610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
13240,24,England,62,66,Hamilton Academical FC,400000.0,1000.0,1481,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
13241,21,Poland,62,72,Śląsk Wrocław,425000.0,1000.0,1692,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
13242,23,Argentina,62,70,Club Atlético Aldosivi,450000.0,2000.0,1663,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
13243,19,England,62,78,Everton,600000.0,5000.0,1328,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
13244,30,Denmark,62,62,Hobro IK,230000.0,2000.0,1244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
13245,21,Brazil,62,69,HJK Helsinki,425000.0,1000.0,1549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

Observamos que esta lista escasa de jugadores contiene excesivos NaN, por lo tanto, es mejor eliminarlos directamente ya que nos darán también problemas con la altura . Además van a perjudicar a nuestro dataset de entrenamiento pues no contienen información.

In [21]:
base = base.dropna(subset = ['Height'])

In [22]:
base["Height"] =base["Height"].apply(height_to_float)
base.head()

C:\phyton\envs\analisi_dades\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Position,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,31,Argentina,94,94,FC Barcelona,110500000.0,565000.0,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,RF,5.70,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,226500000.0
1,33,Portugal,94,94,Juventus,77000000.0,405000.0,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,ST,6.20,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,127100000.0
2,26,Brazil,92,93,Paris Saint-Germain,118500000.0,290000.0,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,LW,5.90,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,228100000.0
3,27,Spain,91,93,Manchester United,72000000.0,260000.0,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,GK,6.40,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,138600000.0
4,27,Belgium,91,92,Manchester City,102000000.0,355000.0,2281,Right,4.0,5.0,4.0,High/ High,Normal,RCM,5.11,154lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,196400000.0


In [23]:
def weight_to_float(x):
    """
    From lbs to float.
    
    """
    if 'lbs' in x:
            ret_val = float(x.replace('lbs', ' '))
    return ret_val

In [24]:
base["Weight"] = base["Weight"].apply(weight_to_float)
base.head()

C:\phyton\envs\analisi_dades\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Position,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,31,Argentina,94,94,FC Barcelona,110500000.0,565000.0,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,RF,5.70,159.0,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,226500000.0
1,33,Portugal,94,94,Juventus,77000000.0,405000.0,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,ST,6.20,183.0,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,127100000.0
2,26,Brazil,92,93,Paris Saint-Germain,118500000.0,290000.0,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,LW,5.90,150.0,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,228100000.0
3,27,Spain,91,93,Manchester United,72000000.0,260000.0,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,GK,6.40,168.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,138600000.0
4,27,Belgium,91,92,Manchester City,102000000.0,355000.0,2281,Right,4.0,5.0,4.0,High/ High,Normal,RCM,5.11,154.0,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,196400000.0


Ahora, comprobamos que existen valores como Nacionalidad,Club,Tipo de Cuerpo, Posición y Pie Preferente que no tienen valores númericos y son valores categóricos. Para ello, aplicaremos la técnica(hot-encoding) usada basándome en la del enunciado con Club y convertiremos los valores a binarios. De esta manera, nos evitaremos problemas con NaNs y podremos convertir los valores no númericos a númericos

In [25]:
wf = base.pop("Nationality")
base = pd.concat([base.reset_index(drop=True), pd.get_dummies(wf, prefix='nat').reset_index(drop=True)],axis=1, sort=False)
filtro_Pos = [col for col in base if col.startswith('nat')]
base.drop(columns=filtro_Pos)

wf = base.pop("Club")
base = pd.concat([base.reset_index(drop=True), pd.get_dummies(wf, prefix='clb').reset_index(drop=True)],axis=1, sort=False)
filtro_Pos = [col for col in base if col.startswith('clb')]
base.drop(columns=filtro_Pos)

wf = base.pop("Body Type")
base = pd.concat([base.reset_index(drop=True), pd.get_dummies(wf, prefix='bdy').reset_index(drop=True)],axis=1, sort=False)
filtro_Pos = [col for col in base if col.startswith('bdy')]
base.drop(columns=filtro_Pos)

wf = base.pop("Position")
base = pd.concat([base.reset_index(drop=True), pd.get_dummies(wf, prefix='pos').reset_index(drop=True)],axis=1, sort=False)
filtro_Pos = [col for col in base if col.startswith('pos')]
base.drop(columns=filtro_Pos)

wf = base.pop("Preferred Foot")
base = pd.concat([base.reset_index(drop=True), pd.get_dummies(wf, prefix='foot').reset_index(drop=True)],axis=1, sort=False)
filtro_Pos = [col for col in base if col.startswith('foot')]
base.drop(columns=filtro_Pos)

base.head()

,Age,Overall,Potential,Value,Wage,Special,International Reputation,Weak Foot,Skill Moves,Work Rate,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,nat_Afghanistan,nat_Albania,nat_Algeria,nat_Andorra,nat_Angola,nat_Antigua & Barbuda,nat_Argentina,nat_Armenia,nat_Australia,nat_Austria,nat_Azerbaijan,nat_Barbados,nat_Belarus,nat_Belgium,nat_Belize,nat_Benin,nat_Bermuda,nat_Bolivia,nat_Bosnia Herzegovina,nat_Botswana,nat_Brazil,nat_Bulgaria,nat_Burkina Faso,nat_Burundi,nat_Cameroon,nat_Canada,nat_Cape Verde,nat_Central African Rep.,nat_Chad,nat_Chile,nat_China PR,nat_Colombia,nat_Comoros,nat_Congo,nat_Costa Rica,nat_Croatia,nat_Cuba,nat_Curacao,nat_Cyprus,nat_Czech Republic,nat_DR Congo,nat_Denmark,nat_Dominican Republic,nat_Ecuador,nat_Egypt,nat_El Salvador,nat_England,nat_Equatorial Guinea,nat_Eritrea,nat_Estonia,nat_Ethiopia,nat_FYR Macedonia,nat_Faroe Islands,nat_Fiji,nat_Finland,nat_France,nat_Gabon,nat_Gambia,nat_Georgia,nat_Germany,nat_Ghana,nat_Greece,nat_Grenada,nat_Guam,nat_Guatemala,nat_Guinea,nat_Guinea Bissau,nat_Guyana,nat_Haiti,nat_Honduras,nat_Hong Kong,nat_Hungary,nat_Iceland,nat_India,nat_Indonesia,nat_Iran,nat_Iraq,nat_Israel,nat_Italy,nat_Ivory Coast,nat_Jamaica,nat_Japan,nat_Jordan,nat_Kazakhstan,nat_Kenya,nat_Korea DPR,nat_Korea Republic,nat_Kosovo,nat_Kuwait,nat_Latvia,nat_Lebanon,nat_Liberia,nat_Libya,nat_Liechtenstein,nat_Lithuania,nat_Luxembourg,nat_Madagascar,nat_Mali,nat_Malta,nat_Mauritania,nat_Mauritius,nat_Mexico,nat_Moldova,nat_Montenegro,nat_Montserrat,nat_Morocco,nat_Mozambique,nat_Namibia,nat_Netherlands,nat_New Caledonia,nat_New Zealand,nat_Nicaragua,nat_Niger,nat_Nigeria,nat_Northern Ireland,nat_Norway,nat_Oman,nat_Palestine,nat_Panama,nat_Paraguay,nat_Peru,nat_Philippines,nat_Poland,nat_Portugal,nat_Puerto Rico,nat_Qatar,nat_Republic of Ireland,nat_Romania,nat_Russia,nat_Rwanda,nat_Saudi Arabia,nat_Scotland,nat_Senegal,nat_Serbia,nat_Sierra Leone,nat_Slovakia,nat_Slovenia,nat_South Africa,nat_South Sudan,nat_Spain,nat_St Kitts Nevis,nat_St Lucia,nat_Sudan,nat_Suriname,nat_Sweden,nat_Switzerland,nat_Syria,nat_São Tomé & Príncipe,nat_Tanzania,nat_Thailand,nat_Togo,nat_Trinidad & Tobago,nat_Tunisia,nat_Turkey,nat_Uganda,nat_Ukraine,nat_United Arab Emirates,nat_United States,nat_Uruguay,nat_Uzbekistan,nat_Venezuela,nat_Wales,nat_Zambia,nat_Zimbabwe,clb_ SSV Jahn Regensburg,clb_1. FC Heidenheim 1846,clb_1. FC Kaiserslautern,clb_1. FC Köln,clb_1. FC Magdeburg,clb_1. FC Nürnberg,clb_1. FC Union Berlin,clb_1. FSV Mainz 05,clb_AC Ajaccio,clb_AC Horsens,clb_AD Alcorcón,clb_ADO Den Haag,clb_AEK Athens,clb_AFC Wimbledon,clb_AIK,clb_AJ Auxerre,clb_AS Béziers,clb_AS Monaco,clb_AS Nancy Lorraine,clb_AS Saint-Étienne,clb_AZ Alkmaar,clb_Aalborg BK,clb_Aarhus GF,clb_Aberdeen,clb_Accrington Stanley,clb_Adelaide United,clb_Ajax,clb_Akhisar Belediyespor,clb_Al Ahli,clb_Al Batin,clb_Al Faisaly,clb_Al Fateh,clb_Al Fayha,clb_Al Hazem,clb_Al Hilal,clb_Al Ittihad,clb_Al Nassr,clb_Al Qadisiyah,clb_Al Raed,clb_Al Shabab,clb_Al Taawoun,clb_Al Wehda,clb_Alanyaspor,clb_Albacete BP,clb_Alianza Petrolera,clb_Amiens SC,clb_América FC (Minas Gerais),clb_América de Cali,clb_Angers SCO,clb_Antalyaspor,clb_Argentinos Juniors,clb_Arka Gdynia,clb_Arsenal,clb_Ascoli,clb_Aston Villa,clb_Atalanta,clb_Athletic Club de Bilbao,clb_Atiker Konyaspor,clb_Atlanta United,clb_Atlético Bucaramanga,clb_Atlético Huila,clb_Atlético Madrid,clb_Atlético Mineiro,clb_Atlético Nacional,clb_Atlético Paranaense,clb_Atlético Tucumán,clb_Audax Italiano,clb_BB Erzurumspor,clb_BK Häcken,clb_BSC Young Boys,clb_Bahia,clb_Barnsley,clb_Bayer

Otros valor no númerico es Work Rate, este se divide según su valoración en ataque y defensa, el valor antes de la barra es el de ataque y el valor de despues de la barra es el de defensa. Sigue este formato --> "Valoración en ataque"/ "Valoracion en Defensa".Este valor puede ser "High","Medium" o "Low".  
Para ello, a través de esta función, dividiremos en dos columnas el valor en ataque y el valor en defensa, de esta manera tendremos más facilidad de tratarlo.

In [26]:
base[['Attack Work Rate','Defense Work Rate']] = base['Work Rate'].str.split('/',n=1,expand= True)
base.pop("Work Rate")
base.head()

,Age,Overall,Potential,Value,Wage,Special,International Reputation,Weak Foot,Skill Moves,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,nat_Afghanistan,nat_Albania,nat_Algeria,nat_Andorra,nat_Angola,nat_Antigua & Barbuda,nat_Argentina,nat_Armenia,nat_Australia,nat_Austria,nat_Azerbaijan,nat_Barbados,nat_Belarus,nat_Belgium,nat_Belize,nat_Benin,nat_Bermuda,nat_Bolivia,nat_Bosnia Herzegovina,nat_Botswana,nat_Brazil,nat_Bulgaria,nat_Burkina Faso,nat_Burundi,nat_Cameroon,nat_Canada,nat_Cape Verde,nat_Central African Rep.,nat_Chad,nat_Chile,nat_China PR,nat_Colombia,nat_Comoros,nat_Congo,nat_Costa Rica,nat_Croatia,nat_Cuba,nat_Curacao,nat_Cyprus,nat_Czech Republic,nat_DR Congo,nat_Denmark,nat_Dominican Republic,nat_Ecuador,nat_Egypt,nat_El Salvador,nat_England,nat_Equatorial Guinea,nat_Eritrea,nat_Estonia,nat_Ethiopia,nat_FYR Macedonia,nat_Faroe Islands,nat_Fiji,nat_Finland,nat_France,nat_Gabon,nat_Gambia,nat_Georgia,nat_Germany,nat_Ghana,nat_Greece,nat_Grenada,nat_Guam,nat_Guatemala,nat_Guinea,nat_Guinea Bissau,nat_Guyana,nat_Haiti,nat_Honduras,nat_Hong Kong,nat_Hungary,nat_Iceland,nat_India,nat_Indonesia,nat_Iran,nat_Iraq,nat_Israel,nat_Italy,nat_Ivory Coast,nat_Jamaica,nat_Japan,nat_Jordan,nat_Kazakhstan,nat_Kenya,nat_Korea DPR,nat_Korea Republic,nat_Kosovo,nat_Kuwait,nat_Latvia,nat_Lebanon,nat_Liberia,nat_Libya,nat_Liechtenstein,nat_Lithuania,nat_Luxembourg,nat_Madagascar,nat_Mali,nat_Malta,nat_Mauritania,nat_Mauritius,nat_Mexico,nat_Moldova,nat_Montenegro,nat_Montserrat,nat_Morocco,nat_Mozambique,nat_Namibia,nat_Netherlands,nat_New Caledonia,nat_New Zealand,nat_Nicaragua,nat_Niger,nat_Nigeria,nat_Northern Ireland,nat_Norway,nat_Oman,nat_Palestine,nat_Panama,nat_Paraguay,nat_Peru,nat_Philippines,nat_Poland,nat_Portugal,nat_Puerto Rico,nat_Qatar,nat_Republic of Ireland,nat_Romania,nat_Russia,nat_Rwanda,nat_Saudi Arabia,nat_Scotland,nat_Senegal,nat_Serbia,nat_Sierra Leone,nat_Slovakia,nat_Slovenia,nat_South Africa,nat_South Sudan,nat_Spain,nat_St Kitts Nevis,nat_St Lucia,nat_Sudan,nat_Suriname,nat_Sweden,nat_Switzerland,nat_Syria,nat_São Tomé & Príncipe,nat_Tanzania,nat_Thailand,nat_Togo,nat_Trinidad & Tobago,nat_Tunisia,nat_Turkey,nat_Uganda,nat_Ukraine,nat_United Arab Emirates,nat_United States,nat_Uruguay,nat_Uzbekistan,nat_Venezuela,nat_Wales,nat_Zambia,nat_Zimbabwe,clb_ SSV Jahn Regensburg,clb_1. FC Heidenheim 1846,clb_1. FC Kaiserslautern,clb_1. FC Köln,clb_1. FC Magdeburg,clb_1. FC Nürnberg,clb_1. FC Union Berlin,clb_1. FSV Mainz 05,clb_AC Ajaccio,clb_AC Horsens,clb_AD Alcorcón,clb_ADO Den Haag,clb_AEK Athens,clb_AFC Wimbledon,clb_AIK,clb_AJ Auxerre,clb_AS Béziers,clb_AS Monaco,clb_AS Nancy Lorraine,clb_AS Saint-Étienne,clb_AZ Alkmaar,clb_Aalborg BK,clb_Aarhus GF,clb_Aberdeen,clb_Accrington Stanley,clb_Adelaide United,clb_Ajax,clb_Akhisar Belediyespor,clb_Al Ahli,clb_Al Batin,clb_Al Faisaly,clb_Al Fateh,clb_Al Fayha,clb_Al Hazem,clb_Al Hilal,clb_Al Ittihad,clb_Al Nassr,clb_Al Qadisiyah,clb_Al Raed,clb_Al Shabab,clb_Al Taawoun,clb_Al Wehda,clb_Alanyaspor,clb_Albacete BP,clb_Alianza Petrolera,clb_Amiens SC,clb_América FC (Minas Gerais),clb_América de Cali,clb_Angers SCO,clb_Antalyaspor,clb_Argentinos Juniors,clb_Arka Gdynia,clb_Arsenal,clb_Ascoli,clb_Aston Villa,clb_Atalanta,clb_Athletic Club de Bilbao,clb_Atiker Konyaspor,clb_Atlanta United,clb_Atlético Bucaramanga,clb_Atlético Huila,clb_Atlético Madrid,clb_Atlético Mineiro,clb_Atlético Nacional,clb_Atlético Paranaense,clb_Atlético Tucumán,clb_Audax Italiano,clb_BB Erzurumspor,clb_BK Häcken,clb_BSC Young Boys,clb_Bahia,clb_Barnsley,clb_Bayer 04 Leverk

Y ahora volveremos a aplicar la técnica utilizada anteriormente para convertir valores categóricos a binarios

In [27]:
wf = base.pop("Attack Work Rate")
base = pd.concat([base.reset_index(drop=True), pd.get_dummies(wf, prefix='AWR').reset_index(drop=True)],axis=1, sort=False)
filtro_Pos = [col for col in base if col.startswith('AWR')]
base.drop(columns=filtro_Pos)

wf = base.pop("Defense Work Rate")
base = pd.concat([base.reset_index(drop=True), pd.get_dummies(wf, prefix='DWR').reset_index(drop=True)],axis=1, sort=False)
filtro_Pos = [col for col in base if col.startswith('DWR')]
base.drop(columns=filtro_Pos)
base.head()

,Age,Overall,Potential,Value,Wage,Special,International Reputation,Weak Foot,Skill Moves,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,nat_Afghanistan,nat_Albania,nat_Algeria,nat_Andorra,nat_Angola,nat_Antigua & Barbuda,nat_Argentina,nat_Armenia,nat_Australia,nat_Austria,nat_Azerbaijan,nat_Barbados,nat_Belarus,nat_Belgium,nat_Belize,nat_Benin,nat_Bermuda,nat_Bolivia,nat_Bosnia Herzegovina,nat_Botswana,nat_Brazil,nat_Bulgaria,nat_Burkina Faso,nat_Burundi,nat_Cameroon,nat_Canada,nat_Cape Verde,nat_Central African Rep.,nat_Chad,nat_Chile,nat_China PR,nat_Colombia,nat_Comoros,nat_Congo,nat_Costa Rica,nat_Croatia,nat_Cuba,nat_Curacao,nat_Cyprus,nat_Czech Republic,nat_DR Congo,nat_Denmark,nat_Dominican Republic,nat_Ecuador,nat_Egypt,nat_El Salvador,nat_England,nat_Equatorial Guinea,nat_Eritrea,nat_Estonia,nat_Ethiopia,nat_FYR Macedonia,nat_Faroe Islands,nat_Fiji,nat_Finland,nat_France,nat_Gabon,nat_Gambia,nat_Georgia,nat_Germany,nat_Ghana,nat_Greece,nat_Grenada,nat_Guam,nat_Guatemala,nat_Guinea,nat_Guinea Bissau,nat_Guyana,nat_Haiti,nat_Honduras,nat_Hong Kong,nat_Hungary,nat_Iceland,nat_India,nat_Indonesia,nat_Iran,nat_Iraq,nat_Israel,nat_Italy,nat_Ivory Coast,nat_Jamaica,nat_Japan,nat_Jordan,nat_Kazakhstan,nat_Kenya,nat_Korea DPR,nat_Korea Republic,nat_Kosovo,nat_Kuwait,nat_Latvia,nat_Lebanon,nat_Liberia,nat_Libya,nat_Liechtenstein,nat_Lithuania,nat_Luxembourg,nat_Madagascar,nat_Mali,nat_Malta,nat_Mauritania,nat_Mauritius,nat_Mexico,nat_Moldova,nat_Montenegro,nat_Montserrat,nat_Morocco,nat_Mozambique,nat_Namibia,nat_Netherlands,nat_New Caledonia,nat_New Zealand,nat_Nicaragua,nat_Niger,nat_Nigeria,nat_Northern Ireland,nat_Norway,nat_Oman,nat_Palestine,nat_Panama,nat_Paraguay,nat_Peru,nat_Philippines,nat_Poland,nat_Portugal,nat_Puerto Rico,nat_Qatar,nat_Republic of Ireland,nat_Romania,nat_Russia,nat_Rwanda,nat_Saudi Arabia,nat_Scotland,nat_Senegal,nat_Serbia,nat_Sierra Leone,nat_Slovakia,nat_Slovenia,nat_South Africa,nat_South Sudan,nat_Spain,nat_St Kitts Nevis,nat_St Lucia,nat_Sudan,nat_Suriname,nat_Sweden,nat_Switzerland,nat_Syria,nat_São Tomé & Príncipe,nat_Tanzania,nat_Thailand,nat_Togo,nat_Trinidad & Tobago,nat_Tunisia,nat_Turkey,nat_Uganda,nat_Ukraine,nat_United Arab Emirates,nat_United States,nat_Uruguay,nat_Uzbekistan,nat_Venezuela,nat_Wales,nat_Zambia,nat_Zimbabwe,clb_ SSV Jahn Regensburg,clb_1. FC Heidenheim 1846,clb_1. FC Kaiserslautern,clb_1. FC Köln,clb_1. FC Magdeburg,clb_1. FC Nürnberg,clb_1. FC Union Berlin,clb_1. FSV Mainz 05,clb_AC Ajaccio,clb_AC Horsens,clb_AD Alcorcón,clb_ADO Den Haag,clb_AEK Athens,clb_AFC Wimbledon,clb_AIK,clb_AJ Auxerre,clb_AS Béziers,clb_AS Monaco,clb_AS Nancy Lorraine,clb_AS Saint-Étienne,clb_AZ Alkmaar,clb_Aalborg BK,clb_Aarhus GF,clb_Aberdeen,clb_Accrington Stanley,clb_Adelaide United,clb_Ajax,clb_Akhisar Belediyespor,clb_Al Ahli,clb_Al Batin,clb_Al Faisaly,clb_Al Fateh,clb_Al Fayha,clb_Al Hazem,clb_Al Hilal,clb_Al Ittihad,clb_Al Nassr,clb_Al Qadisiyah,clb_Al Raed,clb_Al Shabab,clb_Al Taawoun,clb_Al Wehda,clb_Alanyaspor,clb_Albacete BP,clb_Alianza Petrolera,clb_Amiens SC,clb_América FC (Minas Gerais),clb_América de Cali,clb_Angers SCO,clb_Antalyaspor,clb_Argentinos Juniors,clb_Arka Gdynia,clb_Arsenal,clb_Ascoli,clb_Aston Villa,clb_Atalanta,clb_Athletic Club de Bilbao,clb_Atiker Konyaspor,clb_Atlanta United,clb_Atlético Bucaramanga,clb_Atlético Huila,clb_Atlético Madrid,clb_Atlético Mineiro,clb_Atlético Nacional,clb_Atlético Paranaense,clb_Atlético Tucumán,clb_Audax Italiano,clb_BB Erzurumspor,clb_BK Häcken,clb_BSC Young Boys,clb_Bahia,clb_Barnsley,clb_Bayer 04 Leverk

Observamos que deberíamos tratar los valores de rendimiento del jugador en cada posición(LS	ST	RS	LW	LF	CF	RF	RW	LAM	CAM	RAM	LM	LCM	CM	RCM	RM	LWB	LDM	CDM	RDM	RWB	LB	LCB	CB	RCB	RB) . Para ello, simplemte sumamos los valores para que quede en un solo valor númerico.  Con esta función se sumarán los dos valores: 

In [28]:
def add_pos(x):    
    """
    Suma de los valores. 
    
    """
    
    if(x != x):
        return np.nan
    
    values = x.split('+')
    
    return (float(values[0])+float(values[1]))

Utilizamos dicha función para aplicarla a cada posición posible

In [29]:
base["LS"] = base["LS"].apply(add_pos)
base["ST"] = base["ST"].apply(add_pos)
base["RS"] = base["RS"].apply(add_pos)
base["LW"] = base["LW"].apply(add_pos)
base["LF"] = base["LF"].apply(add_pos)
base["CF"] = base["CF"].apply(add_pos)
base["RF"] = base["RF"].apply(add_pos)
base["RW"] = base["RW"].apply(add_pos)
base["LAM"] = base["LAM"].apply(add_pos)
base["CAM"] = base["CAM"].apply(add_pos)
base["RAM"] = base["RAM"].apply(add_pos)
base["LM"] = base["LM"].apply(add_pos)
base["LCM"] = base["LCM"].apply(add_pos)
base["CM"] = base["CM"].apply(add_pos)
base["RCM"] = base["RCM"].apply(add_pos)
base["RM"] = base["RM"].apply(add_pos)
base["LWB"] = base["LWB"].apply(add_pos)
base["LDM"] = base["LDM"].apply(add_pos)
base["CDM"] = base["CDM"].apply(add_pos)
base["RDM"] = base["RDM"].apply(add_pos)
base["RWB"] = base["RWB"].apply(add_pos)
base["LB"] = base["LB"].apply(add_pos)
base["LCB"] = base["LCB"].apply(add_pos)
base["CB"] = base["CB"].apply(add_pos)
base["RCB"] = base["RCB"].apply(add_pos)
base["RB"] = base["RB"].apply(add_pos)
base.head()

,Age,Overall,Potential,Value,Wage,Special,International Reputation,Weak Foot,Skill Moves,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,nat_Afghanistan,nat_Albania,nat_Algeria,nat_Andorra,nat_Angola,nat_Antigua & Barbuda,nat_Argentina,nat_Armenia,nat_Australia,nat_Austria,nat_Azerbaijan,nat_Barbados,nat_Belarus,nat_Belgium,nat_Belize,nat_Benin,nat_Bermuda,nat_Bolivia,nat_Bosnia Herzegovina,nat_Botswana,nat_Brazil,nat_Bulgaria,nat_Burkina Faso,nat_Burundi,nat_Cameroon,nat_Canada,nat_Cape Verde,nat_Central African Rep.,nat_Chad,nat_Chile,nat_China PR,nat_Colombia,nat_Comoros,nat_Congo,nat_Costa Rica,nat_Croatia,nat_Cuba,nat_Curacao,nat_Cyprus,nat_Czech Republic,nat_DR Congo,nat_Denmark,nat_Dominican Republic,nat_Ecuador,nat_Egypt,nat_El Salvador,nat_England,nat_Equatorial Guinea,nat_Eritrea,nat_Estonia,nat_Ethiopia,nat_FYR Macedonia,nat_Faroe Islands,nat_Fiji,nat_Finland,nat_France,nat_Gabon,nat_Gambia,nat_Georgia,nat_Germany,nat_Ghana,nat_Greece,nat_Grenada,nat_Guam,nat_Guatemala,nat_Guinea,nat_Guinea Bissau,nat_Guyana,nat_Haiti,nat_Honduras,nat_Hong Kong,nat_Hungary,nat_Iceland,nat_India,nat_Indonesia,nat_Iran,nat_Iraq,nat_Israel,nat_Italy,nat_Ivory Coast,nat_Jamaica,nat_Japan,nat_Jordan,nat_Kazakhstan,nat_Kenya,nat_Korea DPR,nat_Korea Republic,nat_Kosovo,nat_Kuwait,nat_Latvia,nat_Lebanon,nat_Liberia,nat_Libya,nat_Liechtenstein,nat_Lithuania,nat_Luxembourg,nat_Madagascar,nat_Mali,nat_Malta,nat_Mauritania,nat_Mauritius,nat_Mexico,nat_Moldova,nat_Montenegro,nat_Montserrat,nat_Morocco,nat_Mozambique,nat_Namibia,nat_Netherlands,nat_New Caledonia,nat_New Zealand,nat_Nicaragua,nat_Niger,nat_Nigeria,nat_Northern Ireland,nat_Norway,nat_Oman,nat_Palestine,nat_Panama,nat_Paraguay,nat_Peru,nat_Philippines,nat_Poland,nat_Portugal,nat_Puerto Rico,nat_Qatar,nat_Republic of Ireland,nat_Romania,nat_Russia,nat_Rwanda,nat_Saudi Arabia,nat_Scotland,nat_Senegal,nat_Serbia,nat_Sierra Leone,nat_Slovakia,nat_Slovenia,nat_South Africa,nat_South Sudan,nat_Spain,nat_St Kitts Nevis,nat_St Lucia,nat_Sudan,nat_Suriname,nat_Sweden,nat_Switzerland,nat_Syria,nat_São Tomé & Príncipe,nat_Tanzania,nat_Thailand,nat_Togo,nat_Trinidad & Tobago,nat_Tunisia,nat_Turkey,nat_Uganda,nat_Ukraine,nat_United Arab Emirates,nat_United States,nat_Uruguay,nat_Uzbekistan,nat_Venezuela,nat_Wales,nat_Zambia,nat_Zimbabwe,clb_ SSV Jahn Regensburg,clb_1. FC Heidenheim 1846,clb_1. FC Kaiserslautern,clb_1. FC Köln,clb_1. FC Magdeburg,clb_1. FC Nürnberg,clb_1. FC Union Berlin,clb_1. FSV Mainz 05,clb_AC Ajaccio,clb_AC Horsens,clb_AD Alcorcón,clb_ADO Den Haag,clb_AEK Athens,clb_AFC Wimbledon,clb_AIK,clb_AJ Auxerre,clb_AS Béziers,clb_AS Monaco,clb_AS Nancy Lorraine,clb_AS Saint-Étienne,clb_AZ Alkmaar,clb_Aalborg BK,clb_Aarhus GF,clb_Aberdeen,clb_Accrington Stanley,clb_Adelaide United,clb_Ajax,clb_Akhisar Belediyespor,clb_Al Ahli,clb_Al Batin,clb_Al Faisaly,clb_Al Fateh,clb_Al Fayha,clb_Al Hazem,clb_Al Hilal,clb_Al Ittihad,clb_Al Nassr,clb_Al Qadisiyah,clb_Al Raed,clb_Al Shabab,clb_Al Taawoun,clb_Al Wehda,clb_Alanyaspor,clb_Albacete BP,clb_Alianza Petrolera,clb_Amiens SC,clb_América FC (Minas Gerais),clb_América de Cali,clb_Angers SCO,clb_Antalyaspor,clb_Argentinos Juniors,clb_Arka Gdynia,clb_Arsenal,clb_Ascoli,clb_Aston Villa,clb_Atalanta,clb_Athletic Club de Bilbao,clb_Atiker Konyaspor,clb_Atlanta United,clb_Atlético Bucaramanga,clb_Atlético Huila,clb_Atlético Madrid,clb_Atlético Mineiro,clb_Atlético Nacional,clb_Atlético Paranaense,clb_Atlético Tucumán,clb_Audax Italiano,clb_BB Erzurumspor,clb_BK Häcken,clb_BSC Young Boys,clb_Bahia,clb_Barnsley,clb_Bayer 04 Leverk

Por último, vemos como David de Gea (y el resto de porteros) tiene NaNs en estos valores, para ello podríamos sustuir estos modificadores de posición por ceros, pero no me parece la solución más óptima. Una posible solución, es rellenar las NaNs con las medias de las columnas, no será la solución más eficiente pero es una posible solución alterna a tener que imputar ceros.

In [30]:
base.fillna(base.mean(), inplace=True)
base.head()

,Age,Overall,Potential,Value,Wage,Special,International Reputation,Weak Foot,Skill Moves,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,nat_Afghanistan,nat_Albania,nat_Algeria,nat_Andorra,nat_Angola,nat_Antigua & Barbuda,nat_Argentina,nat_Armenia,nat_Australia,nat_Austria,nat_Azerbaijan,nat_Barbados,nat_Belarus,nat_Belgium,nat_Belize,nat_Benin,nat_Bermuda,nat_Bolivia,nat_Bosnia Herzegovina,nat_Botswana,nat_Brazil,nat_Bulgaria,nat_Burkina Faso,nat_Burundi,nat_Cameroon,nat_Canada,nat_Cape Verde,nat_Central African Rep.,nat_Chad,nat_Chile,nat_China PR,nat_Colombia,nat_Comoros,nat_Congo,nat_Costa Rica,nat_Croatia,nat_Cuba,nat_Curacao,nat_Cyprus,nat_Czech Republic,nat_DR Congo,nat_Denmark,nat_Dominican Republic,nat_Ecuador,nat_Egypt,nat_El Salvador,nat_England,nat_Equatorial Guinea,nat_Eritrea,nat_Estonia,nat_Ethiopia,nat_FYR Macedonia,nat_Faroe Islands,nat_Fiji,nat_Finland,nat_France,nat_Gabon,nat_Gambia,nat_Georgia,nat_Germany,nat_Ghana,nat_Greece,nat_Grenada,nat_Guam,nat_Guatemala,nat_Guinea,nat_Guinea Bissau,nat_Guyana,nat_Haiti,nat_Honduras,nat_Hong Kong,nat_Hungary,nat_Iceland,nat_India,nat_Indonesia,nat_Iran,nat_Iraq,nat_Israel,nat_Italy,nat_Ivory Coast,nat_Jamaica,nat_Japan,nat_Jordan,nat_Kazakhstan,nat_Kenya,nat_Korea DPR,nat_Korea Republic,nat_Kosovo,nat_Kuwait,nat_Latvia,nat_Lebanon,nat_Liberia,nat_Libya,nat_Liechtenstein,nat_Lithuania,nat_Luxembourg,nat_Madagascar,nat_Mali,nat_Malta,nat_Mauritania,nat_Mauritius,nat_Mexico,nat_Moldova,nat_Montenegro,nat_Montserrat,nat_Morocco,nat_Mozambique,nat_Namibia,nat_Netherlands,nat_New Caledonia,nat_New Zealand,nat_Nicaragua,nat_Niger,nat_Nigeria,nat_Northern Ireland,nat_Norway,nat_Oman,nat_Palestine,nat_Panama,nat_Paraguay,nat_Peru,nat_Philippines,nat_Poland,nat_Portugal,nat_Puerto Rico,nat_Qatar,nat_Republic of Ireland,nat_Romania,nat_Russia,nat_Rwanda,nat_Saudi Arabia,nat_Scotland,nat_Senegal,nat_Serbia,nat_Sierra Leone,nat_Slovakia,nat_Slovenia,nat_South Africa,nat_South Sudan,nat_Spain,nat_St Kitts Nevis,nat_St Lucia,nat_Sudan,nat_Suriname,nat_Sweden,nat_Switzerland,nat_Syria,nat_São Tomé & Príncipe,nat_Tanzania,nat_Thailand,nat_Togo,nat_Trinidad & Tobago,nat_Tunisia,nat_Turkey,nat_Uganda,nat_Ukraine,nat_United Arab Emirates,nat_United States,nat_Uruguay,nat_Uzbekistan,nat_Venezuela,nat_Wales,nat_Zambia,nat_Zimbabwe,clb_ SSV Jahn Regensburg,clb_1. FC Heidenheim 1846,clb_1. FC Kaiserslautern,clb_1. FC Köln,clb_1. FC Magdeburg,clb_1. FC Nürnberg,clb_1. FC Union Berlin,clb_1. FSV Mainz 05,clb_AC Ajaccio,clb_AC Horsens,clb_AD Alcorcón,clb_ADO Den Haag,clb_AEK Athens,clb_AFC Wimbledon,clb_AIK,clb_AJ Auxerre,clb_AS Béziers,clb_AS Monaco,clb_AS Nancy Lorraine,clb_AS Saint-Étienne,clb_AZ Alkmaar,clb_Aalborg BK,clb_Aarhus GF,clb_Aberdeen,clb_Accrington Stanley,clb_Adelaide United,clb_Ajax,clb_Akhisar Belediyespor,clb_Al Ahli,clb_Al Batin,clb_Al Faisaly,clb_Al Fateh,clb_Al Fayha,clb_Al Hazem,clb_Al Hilal,clb_Al Ittihad,clb_Al Nassr,clb_Al Qadisiyah,clb_Al Raed,clb_Al Shabab,clb_Al Taawoun,clb_Al Wehda,clb_Alanyaspor,clb_Albacete BP,clb_Alianza Petrolera,clb_Amiens SC,clb_América FC (Minas Gerais),clb_América de Cali,clb_Angers SCO,clb_Antalyaspor,clb_Argentinos Juniors,clb_Arka Gdynia,clb_Arsenal,clb_Ascoli,clb_Aston Villa,clb_Atalanta,clb_Athletic Club de Bilbao,clb_Atiker Konyaspor,clb_Atlanta United,clb_Atlético Bucaramanga,clb_Atlético Huila,clb_Atlético Madrid,clb_Atlético Mineiro,clb_Atlético Nacional,clb_Atlético Paranaense,clb_Atlético Tucumán,clb_Audax Italiano,clb_BB Erzurumspor,clb_BK Häcken,clb_BSC Young Boys,clb_Bahia,clb_Barnsley,clb_Bayer 04 Leverk

## Prediction

Finalmente, ya tenemos el dataset construido con los datos que consideros mas relativos para la variable objetivo "Valor".

In [31]:
val = base.pop("Value")
base

,Age,Overall,Potential,Wage,Special,International Reputation,Weak Foot,Skill Moves,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,nat_Afghanistan,nat_Albania,nat_Algeria,nat_Andorra,nat_Angola,nat_Antigua & Barbuda,nat_Argentina,nat_Armenia,nat_Australia,nat_Austria,nat_Azerbaijan,nat_Barbados,nat_Belarus,nat_Belgium,nat_Belize,nat_Benin,nat_Bermuda,nat_Bolivia,nat_Bosnia Herzegovina,nat_Botswana,nat_Brazil,nat_Bulgaria,nat_Burkina Faso,nat_Burundi,nat_Cameroon,nat_Canada,nat_Cape Verde,nat_Central African Rep.,nat_Chad,nat_Chile,nat_China PR,nat_Colombia,nat_Comoros,nat_Congo,nat_Costa Rica,nat_Croatia,nat_Cuba,nat_Curacao,nat_Cyprus,nat_Czech Republic,nat_DR Congo,nat_Denmark,nat_Dominican Republic,nat_Ecuador,nat_Egypt,nat_El Salvador,nat_England,nat_Equatorial Guinea,nat_Eritrea,nat_Estonia,nat_Ethiopia,nat_FYR Macedonia,nat_Faroe Islands,nat_Fiji,nat_Finland,nat_France,nat_Gabon,nat_Gambia,nat_Georgia,nat_Germany,nat_Ghana,nat_Greece,nat_Grenada,nat_Guam,nat_Guatemala,nat_Guinea,nat_Guinea Bissau,nat_Guyana,nat_Haiti,nat_Honduras,nat_Hong Kong,nat_Hungary,nat_Iceland,nat_India,nat_Indonesia,nat_Iran,nat_Iraq,nat_Israel,nat_Italy,nat_Ivory Coast,nat_Jamaica,nat_Japan,nat_Jordan,nat_Kazakhstan,nat_Kenya,nat_Korea DPR,nat_Korea Republic,nat_Kosovo,nat_Kuwait,nat_Latvia,nat_Lebanon,nat_Liberia,nat_Libya,nat_Liechtenstein,nat_Lithuania,nat_Luxembourg,nat_Madagascar,nat_Mali,nat_Malta,nat_Mauritania,nat_Mauritius,nat_Mexico,nat_Moldova,nat_Montenegro,nat_Montserrat,nat_Morocco,nat_Mozambique,nat_Namibia,nat_Netherlands,nat_New Caledonia,nat_New Zealand,nat_Nicaragua,nat_Niger,nat_Nigeria,nat_Northern Ireland,nat_Norway,nat_Oman,nat_Palestine,nat_Panama,nat_Paraguay,nat_Peru,nat_Philippines,nat_Poland,nat_Portugal,nat_Puerto Rico,nat_Qatar,nat_Republic of Ireland,nat_Romania,nat_Russia,nat_Rwanda,nat_Saudi Arabia,nat_Scotland,nat_Senegal,nat_Serbia,nat_Sierra Leone,nat_Slovakia,nat_Slovenia,nat_South Africa,nat_South Sudan,nat_Spain,nat_St Kitts Nevis,nat_St Lucia,nat_Sudan,nat_Suriname,nat_Sweden,nat_Switzerland,nat_Syria,nat_São Tomé & Príncipe,nat_Tanzania,nat_Thailand,nat_Togo,nat_Trinidad & Tobago,nat_Tunisia,nat_Turkey,nat_Uganda,nat_Ukraine,nat_United Arab Emirates,nat_United States,nat_Uruguay,nat_Uzbekistan,nat_Venezuela,nat_Wales,nat_Zambia,nat_Zimbabwe,clb_ SSV Jahn Regensburg,clb_1. FC Heidenheim 1846,clb_1. FC Kaiserslautern,clb_1. FC Köln,clb_1. FC Magdeburg,clb_1. FC Nürnberg,clb_1. FC Union Berlin,clb_1. FSV Mainz 05,clb_AC Ajaccio,clb_AC Horsens,clb_AD Alcorcón,clb_ADO Den Haag,clb_AEK Athens,clb_AFC Wimbledon,clb_AIK,clb_AJ Auxerre,clb_AS Béziers,clb_AS Monaco,clb_AS Nancy Lorraine,clb_AS Saint-Étienne,clb_AZ Alkmaar,clb_Aalborg BK,clb_Aarhus GF,clb_Aberdeen,clb_Accrington Stanley,clb_Adelaide United,clb_Ajax,clb_Akhisar Belediyespor,clb_Al Ahli,clb_Al Batin,clb_Al Faisaly,clb_Al Fateh,clb_Al Fayha,clb_Al Hazem,clb_Al Hilal,clb_Al Ittihad,clb_Al Nassr,clb_Al Qadisiyah,clb_Al Raed,clb_Al Shabab,clb_Al Taawoun,clb_Al Wehda,clb_Alanyaspor,clb_Albacete BP,clb_Alianza Petrolera,clb_Amiens SC,clb_América FC (Minas Gerais),clb_América de Cali,clb_Angers SCO,clb_Antalyaspor,clb_Argentinos Juniors,clb_Arka Gdynia,clb_Arsenal,clb_Ascoli,clb_Aston Villa,clb_Atalanta,clb_Athletic Club de Bilbao,clb_Atiker Konyaspor,clb_Atlanta United,clb_Atlético Bucaramanga,clb_Atlético Huila,clb_Atlético Madrid,clb_Atlético Mineiro,clb_Atlético Nacional,clb_Atlético Paranaense,clb_Atlético Tucumán,clb_Audax Italiano,clb_BB Erzurumspor,clb_BK Häcken,clb_BSC Young Boys,clb_Bahia,clb_Barnsley,clb_Bayer 04 Leverkusen,c

Now we have only the club information, in hot-econding style. At the same time we have the data we want to predict in a numeric format, the value information. So now we can use this data to train a model. But first we need to split the data to being able to known its true performance.

In [32]:
X_train, X_test, y_train, y_test = train_test_split(base, val, test_size=0.33, random_state=42)

In [33]:
len(X_train)

12166

Now we train a LinearRegression model.

In [34]:
reg = linear_model.LinearRegression().fit(X_train, y_train)

Finally we get a metric $R^2$ for the regression, we use the implementation from <a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html"> sickit-learn</a>.

In [35]:
preds = reg.predict(X_test)

In [36]:
preds[0]

707282.3607195723

In [37]:
y_test[0]

110500000.0

In [38]:
r2_score(y_test, preds)

0.9641759434065622

Obtenemos un 96% de fiabilidad, un valor muy cercano al 100%, lo cual nos demuestra que he contruido el dataset base con datos de entrenamiento relativos a la variable objetivo Valor